**GENERATION DES TITRES**




En général les résumés en une seule ligne sont conçus pour capturer l'idée centrale et les points saillants du contenu original de manière concise. Dans notre contexte ces résumés vont etre considéré comme titres.

Nous verrons comment générer les titres des articles avec simpleT5

**Importation des bibliothèques**

In [ ]:
# install simpleT5
!pip install simplet5
import pandas as pd
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 67.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.8 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6854 sha256=86c7bc2ece215342e11fa5510a2350c7b6e801c4d66353d00cedabfa8db4b7ed
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
  Created wheel

**Lecture des Données :**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Final_dataset.csv")

**Préparation des Données :**

In [ ]:
# simpleT5 expects training and validation dataframes to have 2 columns: "source_text" and "target_text"
df = df[['abstract','title']]
df.columns = ["source_text", "target_text"]

# let's add a prefix to source_text, to uniquely identify kind of task we are performing on the data, in this case --> "summarize"
df['source_text'] = "summarize: "+ df['source_text']

# split the data into training and test
train_df, test_df = train_test_split(df, test_size=0.1)

<ipython-input-3-2b59b6335440>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['source_text'] = "summarize: "+ df['source_text']


**Instanciation et entrainement du Modèle :**

In [ ]:
# instatntiate
model = SimpleT5()
# load
model.from_pretrained("t5","t5-base")
# train
model.train(train_df=train_df, eval_df=test_df, source_max_token_len=512, target_max_token_len=128, max_epochs=5, batch_size=8, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!ls outputs/

simplet5-epoch-0-train-loss-3.0066-val-loss-2.7363
simplet5-epoch-1-train-loss-2.3516-val-loss-2.6718
simplet5-epoch-2-train-loss-2.0017-val-loss-2.7281
simplet5-epoch-3-train-loss-1.6925-val-loss-2.8433
simplet5-epoch-4-train-loss-1.4426-val-loss-2.8909


In [ ]:
# load a trained model
model.load_model("outputs/SimpleT5-epoch-4-train-loss-1.1577", use_gpu=True)
# generate
model.generate("summarize: some text you want to test it on")

AttributeError: ignored

**Génération de Titres :**

In [ ]:
# let's see how it performerd:
sample_abstracts = test_df.sample(2)

for i, abstract in sample_abstracts.iterrows():
    print(f"===== Abstract =====")
    print(abstract['source_text'])
    summary= model.predict(abstract['source_text'])[0]
    print(f"\n===== Actual Title =====")
    print(f"{abstract['target_text']}")
    print(f"\n===== Generated Title =====")
    print(f"{summary}")
    print("\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

===== Abstract =====
summarize: paper analyzes scaling window random csp model ( i.e . model rb ) identify threshold point exactly , denoted $ r_ { cr } $ $ p_ { cr } $ . model , establish scaling window $ w ( n , \delta ) = ( r_ { - } ( n , \delta ) , r_ { + } ( n , \delta ) ) $ probability random instance satisfiable greater $ 1-\delta $ $ r < r_ { - } ( n , \delta ) $ le $ \delta $ $ r > r_ { + } ( n , \delta ) $ . specifically , obtain following result $ $ w ( n , \delta ) = ( r_ { cr } -\theta ( \frac { 1 } { n^ { 1-\epsilon } \ln n } ) , \ r_ { cr } +\theta ( \frac { 1 } { n\ln n } ) ) , $ $ $ 0\leq\epsilon < 1 $ constant . similar result respect parameter $ p $ also obtained . since instance generated model rb shown hard threshold , first attempt , far know , analyze scaling window model hard instance .

===== Actual Title =====
scaling window model rb

===== Generated Title =====
analysis scaling window threshold csp model

 +++++++++++++++++++++++++++++++++++++++++++++++++++++

**Evaluation :**

Parmis les métriques couramment utilisées pour évaluer la qualité des résumés générés on trouve : le score ROUGE (Recall-Oriented Understudy for Gisting Evaluation)

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Sample references (actual titles)
references = test_df['target_text'].tolist()

# Generate summaries for the entire test dataset
generated_summaries = []
for _, abstract in test_df.iterrows():
    generated_summary = model.predict(abstract['source_text'])[0]
    generated_summaries.append(generated_summary)

# Calculate ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1','rougeL'], use_stemmer=True)
rouge_scores = []
for generated, reference in zip(generated_summaries, references):
    rouge = scorer.score(generated, reference)
    rouge_scores.append(rouge)

average_rouge1 = sum(score['rouge1'].fmeasure for score in rouge_scores) / len(rouge_scores)
average_rougeL = sum(score['rougeL'].fmeasure for score in rouge_scores) / len(rouge_scores)

print("Average ROUGE-1 Score:", average_rouge1)
print("Average ROUGE-L Score:", average_rougeL)

Average ROUGE-1 Score: 0.42866724778489473
Average ROUGE-L Score: 0.3848550530006163


Le score ROUGE-1 est relativement élevé, suggérant que les mots individuels (unigrammes) des résumés générés ont une correspondance décente avec les références.
Le score ROUGE-L est relativement élevé, indiquant que les résumés générés partagent des séquences de mots communes avec les références.